In [ ]:
import json
import geemap
import os
import glob
import gdal ## fingers crossed!
import os
import ee
import pandas as pd
import numpy as np
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, Polygon
import folium
import sys


In [ ]:
Map=geemap.Map()

In [ ]:
left_dist=0.1033789690619983
right_dist=0.1033789690619983
upper_dist=0.1402878671830234
lower_dist=0.1402878671830234

In [ ]:
root="/content/drive/MyDrive/NEW_DATASET/"
Sentinel_tif=root+"Sentinel_tif/" 
Sentinel_png=root+"Sentinel_png/"
urban_tif=root+"Urban_tif/"
urban_png=root+"Urban_png/"
temporal_png=root+"Temporal_png/"
temporal_tif=root+"Temporal_tif/"
filename=root+"ACTIVE_LOG.json"
dirs=[root,Sentinel_tif,Sentinel_png,urban_tif,urban_png,temporal_tif,temporal_png]
path = [Sentinel_tif,urban_tif,temporal_tif]

for i in dirs:
  if os.path.exists(i)==False:
    os.mkdir(i)

In [ ]:
dir=[Sentinel_tif,Sentinel_png,urban_tif,urban_png,temporal_tif,temporal_png]
for i in dir:
  dir = os.listdir(i)  
  # Checking if the list is empty or not
  if len(dir) != 0:
    files = glob.glob(i+"*")
    for f in files:
      #os.remove(f)

In [ ]:
def get_boundaries(final_list,c):
  try:
    coordinates = final_list
    def create_polygon(coords, polygon_name):
      ''' Create a polygon from coordinates'''
      polygon = Polygon(coordinates)
      gdf = gpd.GeoDataFrame(crs = {'init' :'epsg:4326'})
      gdf.loc[0,'name'] = polygon_name
      gdf.loc [0, 'geometry'] = polygon
      return gdf
    shapefile = create_polygon(coordinates, "ROI")
    shapefile.to_file("newboundary.shp")
    roi_shp="/content/newboundary.shp"
    roi_shp = geemap.shp_to_ee(roi_shp)
    return get_image(roi_shp,c)
  except Exception as e:
    print(e)
    return -1

In [ ]:
def update_error(c,final_list_string,error):
  data=dict()
  log_p=r"/content/drive/MyDrive/NEW_DATASET/LOG.txt"
  error_p=r"/content/drive/MyDrive/NEW_DATASET/Error.txt"

  f = open(log_p, "a")
  f.write("EXCEPTION"+str(c)+final_list_string+"\n")
  f.close()
  error+=1
  with open(filename) as json_file:
   data = json.load(json_file)
   data["Error_num"]=error
  with open(filename, 'w') as file_object:  #open the file in write mode
   json.dump(data, file_object)
  
  f = open(error_p, "a")
  f.write("EXCEPTION"+str(c)+final_list_string+"\n")
  f.close()

In [ ]:
def get_image(roi,c):
  try:
    sentinel = ee.ImageCollection('COPERNICUS/S2_SR').select(['B2','B4', 'B3', 'B11','B8']) \
                  .filterDate('2017-01-01', '2020-01-30') \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',5)) \
                  .filterBounds(roi).mean()
    sentinel_rgb = sentinel.divide(10000).clip(roi).select(['B4', 'B3', 'B2']).unmask()
    visualization_rgb = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
    }
    #Map.addLayer(sentinel_rgb.clip(roi), visualization_rgb, 'RGB')
    Map.centerObject(roi)
    mask = ee.Image("DLR/WSF/WSF2015/v1").clip(roi)
    visualization_mask = {
    'min': 0,
    'max': 255, 
    }
    mask=mask.expression('b("settlement") == 255 ? 255:0',{'settlement':mask.select('settlement')}).clip(roi)
    #Map.addLayer(mask.clip(roi), visualization_mask, "Human2")



    NDVI = sentinel.normalizedDifference(['B8', 'B4']).rename(['NDVI']);
    MNDWI = sentinel.normalizedDifference(['B3', 'B11']).rename(['MNDWI']); 
    NDBI = sentinel.normalizedDifference(['B11', 'B8']).rename(['NDBI'])
    sentinel=sentinel.addBands(NDBI).addBands(MNDWI).addBands(NDVI)
    image_temporal_mean=sentinel.clip(roi).select(['NDVI','NDBI','MNDWI']).unmask()
    temporal_mean_vis={'bands':['NDBI','NDVI','MNDWI'],'min':-1,'max':1}
    #Map.addLayer(image_temporal_mean.clip(roi), temporal_mean_vis, "Temporal_mean Color")
    return download_image(sentinel_rgb,mask,image_temporal_mean,roi,c)
  except Exception as e:
    print("Error in get_image")
    print(e)
    return -1

In [ ]:
def get_final_list(Lon,Lat):
  try:
    LL=[Lon,Lat]
    UL=[Lon,Lat+left_dist]
    LR=[Lon+lower_dist,Lat]
    UR=[Lon+lower_dist,Lat+left_dist]
    final_list=[LL,LR,UR,UL,LL]
    return final_list
  except Exception as e: 
    print(e)
    print("Error in get_final_list Image")
    return -1

In [ ]:
def download_image(RGB,MASK,TEMPORAL,roi,c):
    try:
      urban=urban_tif+str(c)+".tif"
      rgb=Sentinel_tif+str(c)+".tif"
      temporal=temporal_tif+str(c)+".tif"
      geemap.ee_export_image(MASK, filename=urban, scale=10, region=roi.geometry(), file_per_band=False)
      geemap.ee_export_image(RGB, filename=rgb, scale=10, region=roi.geometry(), file_per_band=False)
      geemap.ee_export_image(TEMPORAL, filename=temporal, scale=10, region=roi.geometry(), file_per_band=False)
      return convert_image(c)
    except Exception as e: 
      print(e)
      print("Error in download_image Image")
      return -1
    

    

In [ ]:
def convert_image(c):
  try:
    urban=urban_tif+str(c)+".tif"
    rgb=Sentinel_tif+str(c)+".tif"
    temporal=temporal_tif+str(c)+".tif"
    urban_download=urban_png+str(c)+".png"
    rgb_download=Sentinel_png+str(c)+".png"
    temporal_download=temporal_png+str(c)+".png"
    options_list_3band = ['-ot Byte','-of PNG','-scale 0 0.3 0 255'] 
    options_list_tempband = ['-ot Byte','-of PNG','-scale -0.6 0.6 0 255'] 

    options_list_1band = ['-ot Byte','-of PNG']
    options_string_RGB = " ".join(options_list_3band)
    options_string_temp=" ".join(options_list_tempband)
    options_string = " ".join(options_list_1band)

    gdal.Translate(urban_download,urban, options=options_string)
    gdal.Translate(rgb_download,rgb, options=options_string_RGB)
    gdal.Translate(temporal_download,temporal, options=options_string_temp)
    xml1=urban_download+".aux.xml"
    xml2=rgb_download+".aux.xml"
    xml3=temporal_download+".aux.xml"

    os.remove(xml1)
    os.remove(xml2)
    os.remove(xml3)
    os.remove(urban)
    os.remove(rgb)
    os.remove(temporal)
    return 1
  except Exception as e:
    print(e)
    print("error in Convert")
    return -1

In [ ]:
final_list=list()
count=0
result=1

#cord=[23.96269296245415, 72.72114537234593]
cord=[23.96269296245415, 72.72114537234593]



data=dict()

with open(filename) as json_file:
    data = json.load(json_file)
Lat=data["Active_latitude"]
i=data["Active_I"]
error=data['Error_num']
c=data["Lat_c"]
Lon=cord[1]

for j in range(i,100):
  Lon=cord[1]
  data=dict()
  with open(filename) as json_file:
      data = json.load(json_file)
  data["Active_latitude"]=Lat
  data["Lat_c"]=c
  data['Active_I']=j
  with open(filename, 'w') as file_object:  #open the file in write mode
   json.dump(data, file_object)

  for k in range(0,100):
    print(Lon,Lat)
    with open(filename) as json_file:
     data = json.load(json_file)
     error=data['Error_num']
    print("Image_number",c)
    final_list=get_final_list(Lon,Lat)
    final_list_string = ' '.join([str(elem) for elem in final_list])
    num=str(c)
    f = open(r"/content/drive/MyDrive/NEW_DATASET/LOG.txt", "a")
    f.write(num+"\n"+final_list_string+"\n")
    f.close()
    print(final_list)
    result=get_boundaries(final_list,c)
    #result=1
    #result=result*-1
    if result==-1: 
     update_error(c,final_list_string,error)

    if result!=-1:
      c+=1


    # Getting the list of directories
    for i in path:
      dir = os.listdir(i)  
      # Checking if the list is empty or not
      if len(dir) != 0:
        files = glob.glob(i+"*")
        for f in files:
          os.remove(f)

    Lon+=lower_dist
  Lat+=left_dist